<a href="https://colab.research.google.com/github/smallriver0316/python-datascience-study/blob/development/recipe_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# refer to https://github.com/fictivekin/openrecipes
!curl -O https://raw.githubusercontent.com/sameergarg/scala-elasticsearch/master/conf/recipeitems-latest.json.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 29.3M  100 29.3M    0     0  28.2M      0  0:00:01  0:00:01 --:--:-- 28.1M


In [ ]:
!gunzip recipeitems-latest.json.gz

gzip: recipeitems-latest.json already exists; do you wish to overwrite (y or n)? y


In [ ]:
import pandas as pd
import numpy as np
from io import StringIO
!pip install fsspec

     |████████████████████████████████| 118 kB 5.2 MB/s 


In [ ]:
with open('recipeitems-latest.json', 'r', encoding='utf-8') as f:
  data = (line.strip() for line in f)
  data_json = "[{0}]".format(','.join(data))
recipes = pd.read_json(StringIO(data_json))

In [ ]:
recipes.shape

(173278, 17)

In [ ]:
# 1行目を抜き出して確認
recipes.iloc[0]

_id                                {'$oid': '5160756b96cc62079cc2db15'}
name                                    Drop Biscuits and Sausage Gravy
ingredients           Biscuits\n3 cups All-purpose Flour\n2 Tablespo...
url                   http://thepioneerwoman.com/cooking/2013/03/dro...
image                 http://static.thepioneerwoman.com/cooking/file...
ts                                             {'$date': 1365276011104}
cookTime                                                          PT30M
source                                                  thepioneerwoman
recipeYield                                                          12
datePublished                                                2013-03-11
prepTime                                                          PT10M
description           Late Saturday afternoon, after Marlboro Man ha...
totalTime                                                           NaN
creator                                                         

In [ ]:
# ingredientsが煩雑な文字列なので、概要を確認
recipes.ingredients.str.len().describe()

count    173278.000000
mean        244.617926
std         146.705285
min           0.000000
25%         147.000000
50%         221.000000
75%         314.000000
max        9067.000000
Name: ingredients, dtype: float64

In [ ]:
# ingredientsの中で最長のものを確認
recipes.name[np.argmax(recipes.ingredients.str.len())]

'Carrot Pineapple Spice &amp; Brownie Layer Cake with Whipped Cream &amp; Cream Cheese Frosting and Marzipan Carrots'

In [ ]:
# 正規表現の利用例
# 朝食用のレシピをカウント
recipes.description.str.contains('[Bb]reakfast').sum()

3524

In [ ]:
# 材料にシナモンを使用しているレシピをカウント
recipes.ingredients.str.contains('[Cc]innamon').sum()

10526

### レシピ推奨システム
#### 材料のリストを与えるとレシピを与えると、それらすべての材料を使用するレシピを見つけるシステムを作る
#### スパイスからのレシピの探索を試みる

In [ ]:
spice_list = ['salt', 'pepper', 'oregano', 'sage', 'parsley', 'rosemary', 'tarragon', 'thyme', 'paprika', 'cumin']

In [ ]:
# re.IGNORECASEは大文字、小文字を区別しないマッチングを行う
import re
is_spice = dict((spice, recipes.ingredients.str.contains(spice, re.IGNORECASE)) for spice in spice_list)
# 中身を確認
dict(list(is_spice.items())[:10])

{'cumin': 0         False
 1         False
 2          True
 3         False
 4         False
           ...  
 173273    False
 173274    False
 173275     True
 173276    False
 173277    False
 Name: ingredients, Length: 173278, dtype: bool, 'oregano': 0         False
 1         False
 2         False
 3         False
 4         False
           ...  
 173273    False
 173274    False
 173275    False
 173276    False
 173277    False
 Name: ingredients, Length: 173278, dtype: bool, 'paprika': 0         False
 1         False
 2         False
 3         False
 4         False
           ...  
 173273    False
 173274    False
 173275     True
 173276    False
 173277    False
 Name: ingredients, Length: 173278, dtype: bool, 'parsley': 0         False
 1         False
 2         False
 3         False
 4         False
           ...  
 173273    False
 173274    False
 173275    False
 173276    False
 173277    False
 Name: ingredients, Length: 173278, dtype: bool, 'pepper': 0      

In [ ]:
spice_df = pd.DataFrame(is_spice)
spice_df.head()

,salt,pepper,oregano,sage,parsley,rosemary,tarragon,thyme,paprika,cumin
0,False,False,False,True,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,True,True,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False


In [ ]:
selection = spice_df.query('parsley & paprika & tarragon')
len(selection)

10

In [ ]:
recipes.name[selection.index]

2069      All cremat with a Little Gem, dandelion and wa...
74964                         Lobster with Thermidor butter
93768      Burton's Southern Fried Chicken with White Gravy
113926                     Mijo's Slow Cooker Shredded Beef
137686                     Asparagus Soup with Poached Eggs
140530                                 Fried Oyster Po’boys
158475                Lamb shank tagine with herb tabbouleh
158486                 Southern fried chicken in buttermilk
163175            Fried Chicken Sliders with Pickles + Slaw
165243                        Bar Tartine Cauliflower Salad
Name: name, dtype: object